<a href="https://colab.research.google.com/github/Snowdenstyll/Lottery/blob/main/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [87]:
df = pd.read_csv("2024.csv")

In [88]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  322 non-null    object
 1   AP        322 non-null    object
 2   N01       322 non-null    int64 
 3   N02       322 non-null    int64 
 4   N03       322 non-null    int64 
 5   N04       322 non-null    int64 
 6   N05       322 non-null    int64 
 7   N06       322 non-null    int64 
 8   N07       322 non-null    int64 
 9   N08       322 non-null    int64 
 10  N09       322 non-null    int64 
 11  N10       322 non-null    int64 
 12  N11       322 non-null    int64 
 13  N12       322 non-null    int64 
 14  N13       322 non-null    int64 
 15  N14       322 non-null    int64 
 16  N15       322 non-null    int64 
 17  N16       322 non-null    int64 
 18  N17       322 non-null    int64 
 19  N18       322 non-null    int64 
 20  N19       322 non-null    int64 
 21  N20       322 no

In [89]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [90]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 5
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [91]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [92]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 32
ltsm_units = 256
dropout = 0.2

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(dropout))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(dropout))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(dropout))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [93]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0004), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.89, random_state=89)

# Train the model
#history = model.fit(X_train, y_train, batch_size=batch_size, epochs=200, verbose=2)
history = model.fit(X_train, y_train, batch_size=30, epochs=600, verbose=2)

Epoch 1/600
2/2 - 14s - loss: 0.9297 - accuracy: 0.0294 - 14s/epoch - 7s/step
Epoch 2/600
2/2 - 0s - loss: 0.9152 - accuracy: 0.0294 - 31ms/epoch - 15ms/step
Epoch 3/600
2/2 - 0s - loss: 0.9033 - accuracy: 0.1471 - 30ms/epoch - 15ms/step
Epoch 4/600
2/2 - 0s - loss: 0.8922 - accuracy: 0.1176 - 31ms/epoch - 15ms/step
Epoch 5/600
2/2 - 0s - loss: 0.8838 - accuracy: 0.0882 - 30ms/epoch - 15ms/step
Epoch 6/600
2/2 - 0s - loss: 0.8742 - accuracy: 0.1471 - 31ms/epoch - 16ms/step
Epoch 7/600
2/2 - 0s - loss: 0.8659 - accuracy: 0.0882 - 31ms/epoch - 15ms/step
Epoch 8/600
2/2 - 0s - loss: 0.8522 - accuracy: 0.1471 - 30ms/epoch - 15ms/step
Epoch 9/600
2/2 - 0s - loss: 0.8487 - accuracy: 0.1765 - 29ms/epoch - 14ms/step
Epoch 10/600
2/2 - 0s - loss: 0.8384 - accuracy: 0.1765 - 28ms/epoch - 14ms/step
Epoch 11/600
2/2 - 0s - loss: 0.8306 - accuracy: 0.1765 - 29ms/epoch - 15ms/step
Epoch 12/600
2/2 - 0s - loss: 0.8152 - accuracy: 0.1765 - 28ms/epoch - 14ms/step
Epoch 13/600
2/2 - 0s - loss: 0.8034 - 

In [94]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

10/10 [==============================] - 3s 7ms/step - loss: 1.3525 - accuracy: 0.1609
Test Loss: 1.3525375127792358, Test Accuracy: 0.16088327765464783
9/9 [==============================] - 0s 7ms/step - loss: 1.5134 - accuracy: 0.0707
Test Loss: 1.5134319067001343, Test Accuracy: 0.07067137956619263


In [95]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [96]:
import datetime
import pytz

# Get the current time in a specific timezone
tz = pytz.timezone('America/New_York')
current_day = datetime.datetime.now(tz)

time_label = "Midday" if (current_day.hour <= 14) else "Evening"

label = current_day.strftime("%Y_%m_%d_") + time_label
model.save(f'{label}.keras')
print("saved model")

saved model


In [97]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-97-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
317,1,2,12,14,17,23,24,28,32,36,38,44,47,51,55,58,59,62,65,69
318,7,12,15,18,19,23,27,30,33,37,38,43,45,46,48,52,53,55,61,68
319,3,9,10,14,17,19,20,26,27,29,35,37,38,42,44,50,56,57,68,69
320,6,13,17,18,21,30,37,39,42,47,48,49,53,57,58,60,61,62,65,69


In [98]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:5]

top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")
top_10_indices = np.argsort(y_pred[0], axis=0)[::-1][:10]
top_10_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_10_numbers.append(number)

print("The top 10 indices:", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers for the lottery game are:  ", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("The winning indices:", winning_indices)
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 [==============================] - 4s 4s/step
The top 5 indices: [ 0 14 16  7  8]
The top 5 numbers: [9, 42, 51, 23, 30]


The top 10 indices: [ 0 14 16  7  8 18 15  9 17  1]
The top 10 numbers: [9, 42, 51, 23, 30, 55, 48, 31, 54, 10]


The predicted numbers for the lottery game are:   [ 3  4  7 11 13 16 20 25 28 31 34 36 40 43 49 51 55 58 62 65]
The actual numbers in the last lottery game were: [ 9 10 12 17 18 19 20 23 30 31 32 34 37 38 42 48 51 54 55 70]


The winning indices: [ 6  9 11 16 18]
winners: [20 31 34 51 55]
0.25


# Predict the next set of numbers

In [99]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 [==============================] - 4s 4s/step
The predicted numbers for the lottery game: [ 3  4  7 11 13 16 20 25 28 31 34 36 40 43 49 51 55 58 62 65]


The top 5 indices [14  0 16 15  2]
The top 5 numbers: [49, 2, 55, 51, 7]


The top 10 indices [14  0 16 15  2 17  1  3 13 18]
The top 10 numbers: [49, 2, 55, 51, 7, 57, 3, 9, 42, 59]


winning indices [ 6  9 11 16 18]
winning numbers [17, 24, 30, 55, 59]


In [100]:
next = df.copy()
next = next.tail(10)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 24ms/step
The top 5 indices [3 2 1 4 0]
The top 5 numbers: [28, 18, 12, 25, 5]


The top 10 indices [ 3  2  1  4  0  5  6  7 11  8]
The top 10 numbers: [28, 18, 12, 25, 5, 26, 27, 29, 40, 29]


winning indices [ 6  9 11 16 18]
winning numbers [27, 28, 40, 48, 50]


The predicted numbers: [[ 5 12 18 28 25 26 27 29 29 28 35 40 42 45 46 48 48 51 50 54]]
